In [1]:
import re
import csv
import numpy as np
import pandas as pd

In [12]:
gridpacks_map = {
'M020':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M020_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M030':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M030_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M040':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M040_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M050':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M050_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M060':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M060_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M070':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M070_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M075':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M075_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M080':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M080_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M085':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M085_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M090':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M090_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M100':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M100_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M110':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M110_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M120':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M120_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M125':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M125_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M130':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M130_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M140':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M140_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M150':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M150_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz',
'M160':'/cvmfs/cms.cern.ch/phys_generator/gridpacks/slc6_amd64_gcc481/13TeV/madgraph/V5_2.4.2/ChargedHiggsToCB/v1/ChargedHiggsToCB_M160_slc6_amd64_gcc481_CMSSW_7_1_30_tarball.tar.xz'
}

LHEproducer = """import FWCore.ParameterSet.Config as cms

externalLHEProducer = cms.EDProducer("ExternalLHEProducer",
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)

#Link to datacards:
#https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/ChargedHiggsToCB
"""

Pythia8Fragment="""## Pythia8 fragment for 13TeV collisions
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *

generator = cms.EDFilter("Pythia8HadronizerFilter",
    pythiaHepMCVerbosity = cms.untracked.bool(False),
    maxEventsToPrint = cms.untracked.int32(0),
    pythiaPylistVerbosity = cms.untracked.int32(1),
    filterEfficiency = cms.untracked.double(1.0),
    crossSection = cms.untracked.double(0.0),
    comEnergy = cms.double(13000.0),
    PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        processParameters = cms.vstring(
            
#PSweights
        'UncertaintyBands:doVariations = on',
# 3 sets of variations for ISR&FSR up/down
# Reduced sqrt(2)/(1/sqrt(2)), Default 2/0.5 and Conservative 4/0.25 variations
        'UncertaintyBands:List = {\
isrRedHi isr:muRfac=0.707,fsrRedHi fsr:muRfac=0.707,isrRedLo isr:muRfac=1.414,fsrRedLo fsr:muRfac=1.414,\
isrDefHi isr:muRfac=0.5, fsrDefHi fsr:muRfac=0.5,isrDefLo isr:muRfac=2.0,fsrDefLo fsr:muRfac=2.0,\
isrConHi isr:muRfac=0.25, fsrConHi fsr:muRfac=0.25,isrConLo isr:muRfac=4.0,fsrConLo fsr:muRfac=4.0}',
 
        'UncertaintyBands:MPIshowers = on',
        'UncertaintyBands:overSampleFSR = 10.0',
        'UncertaintyBands:overSampleISR = 10.0',
        'UncertaintyBands:FSRpTmin2Fac = 20',
        'UncertaintyBands:ISRpTmin2Fac = 1'
        
        ),
        parameterSets = cms.vstring(
            'pythia8CommonSettings',
            'pythia8CP5Settings',
            'processParameters'
        )
    )
)

ProductionFilterSequence = cms.Sequence(generator)
"""

general_dataset_name = "ChargedHiggsToCB_VALUE_TuneCP5_PSweights_13TeV-madgraph-pythia8"
num_events = 300000

In [13]:
prepid_Map = {
'M020':'HIG-RunIIFall17wmLHEGS-03436', 
'M030':'HIG-RunIIFall17wmLHEGS-03437', 
'M040':'HIG-RunIIFall17wmLHEGS-03438', 
'M050':'HIG-RunIIFall17wmLHEGS-03439', 
'M060':'HIG-RunIIFall17wmLHEGS-03440', 
'M070':'HIG-RunIIFall17wmLHEGS-03441', 
'M075':'HIG-RunIIFall17wmLHEGS-03442', 
'M080':'HIG-RunIIFall17wmLHEGS-03443', 
'M085':'HIG-RunIIFall17wmLHEGS-03444', 
'M090':'HIG-RunIIFall17wmLHEGS-03445', 
'M100':'HIG-RunIIFall17wmLHEGS-03446', 
'M110':'HIG-RunIIFall17wmLHEGS-03447', 
'M120':'HIG-RunIIFall17wmLHEGS-03448', 
'M125':'HIG-RunIIFall17wmLHEGS-03449', 
'M130':'HIG-RunIIFall17wmLHEGS-03450', 
'M140':'HIG-RunIIFall17wmLHEGS-03451', 
'M150':'HIG-RunIIFall17wmLHEGS-03452', 
'M160':'HIG-RunIIFall17wmLHEGS-03453' 
}

In [14]:
MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")

with open('ChargedHiggsToCB_MC2017.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(['Dataset name','Events', 'fragment','notes','Generator', 'prepid'])
    
    for value, path in gridpacks_map.items():
        #print('\t %s - %s' %(key.replace('VALUE',value),path))
        version = MadgraphVersion.search(path).group(0)
        dataset_name = general_dataset_name.replace('VALUE',value)
        events=num_events
        final_fragment = LHEproducer.replace('__GRIDPACK__',path) + '\n' + Pythia8Fragment
        note = dataset_name.replace('_',' ')
        generators="Madgraph_"+str(version)+" Pythia8"
        prepid = prepid_Map[value]
        csvwriter.writerow([dataset_name, events, final_fragment, note, generators, prepid])